In [ ]:
import os
import cv2
from PIL import Image 
import numpy as np 
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_extraction import image as skimg
from skimage.util import view_as_windows
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
import random
import tensorflow as tf
import xmltodict

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, Activation, Cropping2D, ZeroPadding2D, Concatenate
from keras.layers import concatenate, BatchNormalization, Dropout, Lambda
from keras.applications import ResNet50, VGG16, VGG19
from keras import backend as K
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm
import csv
import pandas as pd
#import xmltodict

import gc
#from torch.nn.utils import spectral_norm

Constatns

In [ ]:
human_dataset_folder = r'.\Datasets\dataset_human_white\LABELED\LABELED\images\classified_humans'
human_labels_folder = r'.\Datasets\dataset_human_white\LABELED\LABELED\labels\classified_humans'

dataset_4k_folder = r'.\Datasets\drone_dataset_human_4k\ntut_drone_train\ntut_drone_train\Drone_005\vott-csv-export'
csv_folder = r'.\Datasets\drone_dataset_human_4k\labels'

snow_dataset_folder = r'.\Datasets\other_human_archive\images\train'
snow_xml_folder = r'.\Datasets\other_human_archive\XML_annotations'

minmaxscaler = MinMaxScaler()
IMG_SIZE = (512, 512) #(1024,1024)

## Processing images and mask files

Processing LaDD dataset

In [ ]:
#LaDD dataset

# Define the constants
image_dataset = []
mask_dataset = []
images_fold = snow_dataset_folder
labels_fold = snow_xml_folder
scale = 2

df = pd.read_csv(csv_folder+"\Drone_005-export.csv")

def preprocess_mask(img_shape, directory, filename):
    label_file = os.path.join(directory, filename.replace('.jpg','.xml'))
    image_size = (img_shape[0], img_shape[1])
    mask_image = np.zeros(image_size, dtype=np.uint8)
    if os.path.isfile(label_file):
        with open(label_file, "r") as f:
            xml_content = f.read()
            # Convert the XML string to a dictionary
            xml_dict = xmltodict.parse(xml_content)
            if (isinstance(xml_dict["annotation"]["object"], list)):
                for i in range(len(xml_dict["annotation"]["object"])):
                    x = np.float32(xml_dict["annotation"]["object"][i]["bndbox"]["xmin"])
                    y = np.float32(xml_dict["annotation"]["object"][i]["bndbox"]["ymin"])
                    width = np.float32(xml_dict["annotation"]["object"][i]["bndbox"]["xmax"])
                    height = np.float32(xml_dict["annotation"]["object"][i]["bndbox"]["ymax"])
                    x1 = int(x//scale)
                    y1 = int(y//scale)
                    x2 = int(width//scale)
                    y2 = int(height//scale)
                    cv2.rectangle(mask_image, (x1, y1), (x2, y2), color=255, thickness=-1)
            else:
                x = np.float32(xml_dict["annotation"]["object"]["bndbox"]["xmin"])
                y = np.float32(xml_dict["annotation"]["object"]["bndbox"]["ymin"])
                width = np.float32(xml_dict["annotation"]["object"]["bndbox"]["xmax"])
                height = np.float32(xml_dict["annotation"]["object"]["bndbox"]["ymax"])
                x1 = int(x//scale)
                y1 = int(y//scale)
                x2 = int(width//scale)
                y2 = int(height//scale)
                cv2.rectangle(mask_image, (x1, y1), (x2, y2), color=255, thickness=-1)
    mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2RGB)
    return mask_image

for filename in os.listdir(images_fold):
    if filename.endswith(".jpg"):
        image_file = os.path.join(images_fold, filename)
        image = cv2.imread(image_file)
        image = cv2.resize(image, [image.shape[1]//scale,image.shape[0]//scale])
        if image is not None:
            images = [cv2.cvtColor(image, cv2.COLOR_BGR2RGB), preprocess_mask(image.shape, labels_fold, filename)]
            for ind in range(2):
                size_x = (images[ind].shape[1] // IMG_SIZE) * IMG_SIZE
                size_y = (images[ind].shape[0] // IMG_SIZE) * IMG_SIZE
                patched_images = view_as_windows(images[ind].astype(np.float32), (IMG_SIZE, IMG_SIZE, 3), step=IMG_SIZE)
                patched_images = patched_images.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
                for patch in patched_images:
                    patch = minmaxscaler.fit_transform(patch.reshape(-1, patch.shape[-1])).reshape(patch.shape)
                    if ind is 0:
                        image_dataset.append(patch)
                    else:
                        mask_dataset.append(patch)
print(i)
print(len(image_dataset))
print(len(mask_dataset))

20
1064
1064


Processing dataset Human in white clothes

In [ ]:
#White human dataset
# Define the constants
image_types = {"images": "jpg", "masks": "txt"} # The image types and extensions
image_dataset = []
mask_dataset = []
images_fold = human_dataset_folder
labels_fold = human_labels_folder

def preprocess_mask(img_shape, directory, filename):
    label_file = os.path.join(directory, filename.replace('.jpg','.txt'))
    image_size = (img_shape[0], img_shape[1])
    mask_image = np.zeros(image_size, dtype=np.uint8)
    if os.path.isfile(label_file):
        with open(label_file, "r") as f:
            lines = f.readlines()
        for line in lines:
            class_id, x, y, width, height = map(float, line.split())
            x1 = int((x - width / 2) * image_size[1])
            y1 = int((y - height / 2) * image_size[0])
            x2 = int((x + width / 2) * image_size[1])
            y2 = int((y + height / 2) * image_size[0])
            cv2.rectangle(mask_image, (x1, y1), (x2, y2), color=255, thickness=-1)
    mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2RGB)
    return mask_image

for filename in os.listdir(images_fold):
    if filename.endswith(".jpg"):
        image_file = os.path.join(images_fold, filename)
        image = cv2.imread(image_file)
        if image is not None:
            images = [cv2.cvtColor(image, cv2.COLOR_BGR2RGB), preprocess_mask(image.shape, labels_fold, filename.replace('.jpg','.txt'))]
            for ind in range(2):
                size_x = (images[ind].shape[1] // IMG_SIZE) * IMG_SIZE
                size_y = (images[ind].shape[0] // IMG_SIZE) * IMG_SIZE
                patched_images = view_as_windows(images[ind].astype(np.float32), (IMG_SIZE, IMG_SIZE, 3), step=IMG_SIZE)
                patched_images = patched_images.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
                for patch in patched_images:
                    patch = minmaxscaler.fit_transform(patch.reshape(-1, patch.shape[-1])).reshape(patch.shape)
                    if ind is 0:
                        image_dataset.append(patch)
                    else:
                        mask_dataset.append(patch)
print(i)
print(len(image_dataset))
print(len(mask_dataset))

500
4164
4164


Processing dataset 4k human

In [ ]:
#4k dataset

# Define the constants
image_dataset = []
mask_dataset = []
images_fold = dataset_4k_folder
labels_fold = csv_folder
scale = 4

df = pd.read_csv(csv_folder+"\Drone_005-export.csv")

def preprocess_mask(img_shape, filename):
    label_lines = df[df['image']==filename]
    image_size = (img_shape[0], img_shape[1])
    mask_image = np.zeros(image_size, dtype=np.uint8)
    if label_lines.empty is not True: 
            for index, row in label_lines.iterrows():
                x, y, width, height,  = map(float, row[1:5])
                x1 = int(x//scale)
                y1 = int(y//scale)
                x2 = int(width//scale)
                y2 = int(height//scale)
                cv2.rectangle(mask_image, (x1, y1), (x2, y2), color=255, thickness=-1)
    mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2RGB)
    return mask_image

for filename in os.listdir(images_fold):
    #print(filename)
    if filename.endswith(".jpg"):
        image_file = os.path.join(images_fold, filename)
        image = cv2.imread(image_file)
        image = cv2.resize(image, [image.shape[1]//scale,image.shape[0]//scale])
        #print(image.shape)
        if image is not None:
            images = [cv2.cvtColor(image, cv2.COLOR_BGR2RGB), preprocess_mask(image.shape, filename.replace('.jpg',''))]
            for ind in range(2):
                size_x = (images[ind].shape[1] // IMG_SIZE) * IMG_SIZE
                size_y = (images[ind].shape[0] // IMG_SIZE) * IMG_SIZE
                patched_images = view_as_windows(images[ind].astype(np.float32), (IMG_SIZE, IMG_SIZE, 3), step=IMG_SIZE)
                patched_images = patched_images.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
                for patch in patched_images:
                    patch = minmaxscaler.fit_transform(patch.reshape(-1, patch.shape[-1])).reshape(patch.shape)
                    if ind is 0:
                        image_dataset.append(patch)
                    else:
                        mask_dataset.append(patch)
print(i)
print(len(image_dataset))
print(len(mask_dataset))

Making classes based on mask colors

In [ ]:
class_human = np.array(tuple([255,255,255]))
print(class_human)

class_unlabeled = np.array(tuple([0,0,0]))
print(class_unlabeled)

[255 255 255]
[0 0 0]


In [ ]:
label = patch

In [ ]:
def rgb_to_label(label):
  label_segment = np.zeros(label.shape, dtype=np.uint8)
  label_segment[np.all(label == class_human, axis=-1)] = 0
  label_segment[np.all(np.uint8(label*255) == class_unlabeled, axis=-1)] = 1
  #label_segment[np.all(np.uint8(label*255) == class_test, axis=-1)] = 2
  label_segment = label_segment[:,:,0]
  return label_segment

In [ ]:
labels = []
for i in range(mask_dataset.shape[0]):
  label = rgb_to_label(mask_dataset[i])
  labels.append(label)
labels = np.array(labels)
labels = np.expand_dims(labels, axis=3)
total_classes = len(np.unique(labels))

Splitting dataset

In [ ]:
master_trianing_dataset = image_dataset
X_train, X_test, y_train, y_test = train_test_split(master_trianing_dataset, labels_categorical_dataset, test_size=0.15, random_state=100)


Creating model

In [ ]:
def group_conv_block(inputs, filters, activation='relu', batch_norm=True, groups=32):
    channel_axis = -1
    input_channels = inputs.shape[channel_axis]
    group_size = input_channels // groups
    group_list = []

    for i in range(groups):
        group = inputs[:, :, :, i * group_size:(i + 1) * group_size]
        group = Conv2D(filters // groups, (3, 3), padding='same')(group)
        if batch_norm:
            group = BatchNormalization()(group)
        group = Activation(activation)(group)
        group_list.append(group)

    x = Concatenate(axis=channel_axis)(group_list)
    return x

def custom_residual_block(inputs, filters):
    x = group_conv_block(inputs, filters)
    x = group_conv_block(x, filters)
    shortcut = Conv2D(filters, (1, 1), padding='same')(inputs)
    x = Add()([x, shortcut])
    return x

def unet(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    
    resnet = ResNet50(include_top=False, weights='imagenet', input_tensor=inputs)
    encoder_layers = [resnet.get_layer(layer_name).output for layer_name in ['conv1_relu', 'conv2_block3_out', 'conv3_block4_out', 'conv4_block6_out']]
    
    x = resnet.get_layer('conv5_block3_out').output
    x = custom_residual_block(x, 2048)
    x = Dropout(0.5)(x)

    filters = 1024
    for i in range(4):
        x = UpSampling2D((2, 2))(x)
        x = Concatenate()([x, encoder_layers[-(i+1)]])
        x = custom_residual_block(x, filters)
        x = Dropout(0.5)(x)
        filters //= 2

    x = UpSampling2D((2, 2))(x)

    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(x)

    return tf.keras.Model(inputs=inputs, outputs=outputs)



input_shape = (256, 256, 3)
n_classes = total_classes
model = unet(input_shape, n_classes)

# Fine-tune the pre-trained ResNet50 layers
for layer in model.layers[:len(model.layers) - 1]:
    layer.trainable = True

Defining model parameters

In [ ]:
def jaccard_coef(y_true, y_pred):
  y_true_flatten = K.flatten(y_true)
  y_pred_flatten = K.flatten(y_pred)
  intersection = K.sum(y_true_flatten * y_pred_flatten)
  final_coef_value = (intersection + 1.0) / (K.sum(y_true_flatten) + K.sum(y_pred_flatten) - intersection + 1.0)
  return final_coef_value

In [ ]:
weights = [0.5, 0.5]
dice_loss = sm.losses.DiceLoss(class_weights = weights)
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

In [ ]:
from keras import optimizers, callbacks
metrics = ["accuracy", jaccard_coef]
EPOCHS = 120
LR = 0.0002
optim = optimizers.Adam(LR)

In [ ]:
model.compile(optim, loss=total_loss, metrics=metrics)

Defining callbacks, saving best model and reducing learning rate while training stuck on the same loss

In [ ]:
callbacks = [
    callbacks.ModelCheckpoint('./best_model_ternaus.h5', save_weights_only=True, save_best_only=True, mode='min'),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, min_lr=1e-9, verbose=1),
]
model.optimizer.lr.assign(2e-4)

Check if training on gpu is available and prevent memory from overflowing after training

In [ ]:
print(tf.config.list_physical_devices('GPU')) 

# Check if a GPU is available
if tf.config.list_physical_devices('GPU'):
    print('GPU is available')
else:
    print('GPU is not available')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
GPU is available


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

Visualazing model

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file="satellite_model_plot3.png", show_shapes=True, show_layer_names=True)    

Model training

In [ ]:
import wandb
from wandb.keras import WandbCallback
!wandb login "hidden, due to github publicity"

In [ ]:
wandb.init(project="human_segmentation_lame")

In [ ]:
model_history = model.fit(X_train, y_train,
                          batch_size=4,
                          verbose=1,
                          epochs=120,
                          validation_data=(X_test, y_test),
                          callbacks = [callbacks, WandbCallback()],
                          shuffle=False)

Training statistics

In [ ]:
history_a = model_history
history_a.history

In [ ]:
loss = history_a.history['loss']
val_loss = history_a.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label="Training Loss")
plt.plot(epochs, val_loss, 'r', label="Validation Loss")
plt.title("Training Vs Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
jaccard_coef = history_a.history['iou_score']
val_jaccard_coef = history_a.history['val_iou_score']

epochs = range(1, len(jaccard_coef) + 1)
plt.plot(epochs, jaccard_coef, 'y', label="Training IoU")
plt.plot(epochs, val_jaccard_coef, 'r', label="Validation IoU")
plt.title("Training Vs Validation IoU")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()